In [ ]:
import PyPDF2
import codecs
import codecs, io
import glob
import pandas as pd
import re
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import requests
import json
import time
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:
# Ansichtsoptionen für Pandas Dataframes

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
# Haltestellendaten der Deutschen Bahn AG / DB Station&Service AG 
# Quelle: https://data.deutschebahn.com/dataset/data-haltestellen/resource/27ce767b-4d62-4806-8922-908d2cf65f1e
# Lizenz:  Creative Commons Attribution 4.0 International (CC BY 4.0)

stations = pd.read_csv('D_Bahnhof_2017_09.csv', sep=";", decimal=",")
stations = stations.drop(['DS100', 'IFOPT', 'VERKEHR', 'STATUS'], axis=1)

In [ ]:
#stations.head(5)

In [ ]:
# Alle PDFs im Unterverzeichnis 'tickets' in eine Liste laden

extension = 'pdf'
all_filenames = [i for i in glob.glob('tickets/*.{}'.format(extension))]

In [ ]:
# Parsen der PDFs

list_of_data = []


for t in all_filenames:

    # PDF öffnen
    pdfFileObj = io.open(t, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    
    # 1 Seite des PDFs öffnen - bei manchen Tickets gibt es mehrere Seiten - Funktion folgt
    pageObj = pdfReader.getPage(0)
    
    # Text extrahieren
    text2 = pageObj.extractText()
    pdfFileObj.close()
    
    # Text in einzelne Zeilen  
    values = text2.split('\n');
    data = []
    try:
        # Es gibt verschiedene Ticket-Varianten. Hier wird versucht, die meisten Möglichkeiten abzufangen
        # Die Varianten werden mit 1, 2, 3a-c, 4 dargestellt
        if values[2].split(' ')[1] == 'Hinfahrt:':
            data.append(t.split('\\')[1])
            data.append(str(values[2].split(': ')[1]))
            data.append(values[7])
            data.append(values[8])
            if values[8][-10:-1] == 'R•ckfahrt':
                data.append(values[9].split(' ')[0])
                data.append(values[9].split(' ')[2])
            else:
                data.append(values[10])
                data.append(values[11])
            data.append("1")
        elif values[2].split(' ')[0] == 'G•ltigkeit:':
            data.append(t.split('\\')[1])
            data.append(str((values[3].split(' ')[1][0:10])))
            data.append(values[7].split(' ')[0])
            data.append(values[7].split(' ')[2])
            data.append('-')
            data.append('-')
            data.append("2")
        elif values[2].split(' ')[1] == 'am':
            
            data.append(t.split('\\')[1])
            data.append(str((values[2].split(' ')[2])))
            data.append(values[5].split(' ')[0])
            if values[6][0:3] == 'VIA':
                data.append(values[5].split(' ')[2])
                variante = "3a"
            elif values[4][-9:-1] == 'Hinfahrt':
                data.append(values[5].split(' ')[2])
                variante = "3b"
            else:
                data.append(values[6].split(' ')[1])
                variante = "3c"
            data.append('-')
            data.append('-')
            data.append(variante)
            
        elif values2[6].endswith('Hinfahrt:') == True:
            data.append(t.split('\\')[1])
            data.append(values2[3].split(' ')[1][0:10])
            data.append('-')
            data.append('-')         
            data.append('-')
            data.append('-') 
            data.append("4")
        #print (filename,datum, von1, nach1, von2, nach2)
        #print (filename, datum, von1, nach1, von2, nach2, variante)
        list_of_data.append(data)
    
    except:
        pass

In [ ]:
# Erzeugen des Dataframes

df = pd.DataFrame(list_of_data, columns = ['Datei','Datum', 'Von_1', 'Nach_1', 'Von_2', 'Nach_2', 'Variante'])

In [ ]:
# Die Texte sind etwas "messy" und enthalten falsche Umlaute.
# Hier wird versucht, ein wenig zu "putzen"

df = df.replace({'\+City': ''}, regex=True)
df = df.replace({'\+City, mit IC/ECR•ckfahrt:': ''}, regex=True) 
df = df.replace({'mit IC\/ECR•ckfahrt::': ''}, regex=True)
df = df.replace({'mit ICER•ckfahrt:': ''}, regex=True)
df = df.replace({'mit IC\/ECR•ckfahrt:': ''}, regex=True)
df = df.replace({'K‡ln': 'Köln'}, regex=True)
df = df.replace({'K•ln': 'Köln'}, regex=True)
df = df.replace({'M•nster': 'Münster'}, regex=True)
df = df.replace({'\†': 'ü'}, regex=True)
df = df.replace({'K…ln': 'Köln'}, regex=True)
df = df.replace({'K—ln': 'Köln'}, regex=True)
df = df.replace({'\,': ''}, regex=True)
df = df.replace({'über:': ''}, regex=True)
df['Von_1'] = df['Von_1'].str.strip()
df['Nach_1'] = df['Nach_1'].str.strip()
df['Von_2'] = df['Von_2'].str.strip()
df['Nach_2'] = df['Nach_2'].str.strip()
df['Von_1_bhf'] = df['Von_1'] + ' Hbf'
df['Nach_1_bhf'] = df['Nach_1'] + ' Hbf'
df['Von_2_bhf'] = df['Von_2'] + ' Hbf'
df['Nach_2_bhf'] = df['Nach_2'] + ' Hbf'

In [ ]:
# Funktion, um den Bahnhof mit dem ähnlichen Namen aus der Stationsliste zu finden

def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):

    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    
    col = key1 + '_matches'
    df_1[col] = m2

    return df_1

In [ ]:
# Abgleichen der Haltestellen mit der Stationsliste

df = fuzzy_merge(df, stations, 'Von_1_bhf', 'NAME', threshold=80)

df = fuzzy_merge(df, stations, 'Nach_1_bhf', 'NAME', threshold=80)

df['Von_1_bhf_matches'] = df['Von_1_bhf_matches'].str.split(",", n = 1, expand = True)[0]

df['Nach_1_bhf_matches'] = df['Nach_1_bhf_matches'].str.split(",", n = 1, expand = True)[0]

In [ ]:
# Lat/Long - Daten aus dem Bahn-Datensatz verknüpfen 
# Erst den VON-Bahnhof
df = df.merge(stations, left_on='Von_1_bhf', right_on='NAME', how='outer')

# Spalten umbenennen
df.rename(columns={'LAENGE': 'Von_1_bhf_laenge', 'BREITE': 'Von_1_bhf_breite', 'EVA_NR': 'Von_1_bhf_EVA_NR'}, inplace=True)

# Dann NACH-Bahnhof
df = df.merge(stations, left_on='Nach_1_bhf', right_on='NAME', how='outer')
df.rename(columns={'LAENGE': 'Nach_1_bhf_laenge', 'BREITE': 'Nach_1_bhf_breite', 'EVA_NR': 'Nach_1_bhf_EVA_NR'}, inplace=True)


In [ ]:
# Alternative: Lat/Long Berechnung über Nominatim

#locator = Nominatim(user_agent="bahnticket_auswertung")

#from geopy.extra.rate_limiter import RateLimiter
#geocode = RateLimiter(locator.geocode, min_delay_seconds=3)
#df['Von_1_ltlg'] = df['Von_1_bhf'].apply(geocode)
#df['Von_1_point'] = df['Von_1_ltlg'].apply(lambda loc: tuple(loc.point) if loc else None)
#df['Nach_1_ltlg'] = df['Nach_1_bhf'].apply(geocode)
#df['Nach_1_point'] = df['Nach_1_ltlg'].apply(lambda loc: tuple(loc.point) if loc else None)

In [ ]:
# Ist Rückfahrt gleich Hinfahrt?

df['rueckfahrt_gleich'] = df['Von_1'] == df['Nach_2']

In [ ]:
# Leere und fehlerhafte Zeilen löschen

df = df.dropna(thresh=10)

In [ ]:
# Lat/Long der Bahnhöfe in eine URL-Spalte bauen

df['url'] = 'https://brouter.damsy.net/api/brouter?lonlats=' + df['Von_1_bhf_laenge'].map(str) + ',' + df['Von_1_bhf_breite'].map(str) + '|' + df['Nach_1_bhf_laenge'].map(str) + ',' + df['Nach_1_bhf_breite'].map(str) + '&profile=rail&alternativeidx=0&format=geojson' 

In [ ]:
# Funktion zum Aufrufen der Distanzmessungs-API
# Bitte die Anzahl der Anfragen begrenzen!

def entfernnung_messen(df_url):
    r = requests.get(df_url)
    try:
        json_data = json.loads(r.text)
        ergebnis = json_data['features'][0]['properties']['track-length']
    except:
        ergebnis = 0
        pass
    
    # Pause, um den Server nicht zu überlasten
    time.sleep(2)
    return (ergebnis)

In [ ]:
# Funktion auf ganzen Dataframe anwenden
df['distance'] = df['url'].apply(entfernnung_messen)

In [ ]:
# Ergebnis in Zahl umwandeln und in Kilometern ausgeben
df['distance'] = pd.to_numeric(df['distance'])
df['distance'] = df['distance'] / 1000

In [ ]:
# Ist die Zugstrecke Hin- und Rückfahrt? Dann doppelte Distanz

df['distance_complete'] = np.where(df['rueckfahrt_gleich'] == True,
                                           df['distance'] * 2,
                                           df['distance'])

In [ ]:
# Distanz ausrechnen

summe = df.distance_complete.sum()

In [ ]:
print ('Sie sind', round(summe,0), 'Kilometer mit der Bahn gefahren')